### HW 4
1. **Обучить классификатор**:
* Разбить train.csv на train и valid выборки
* Обучить модель, которая по тексту цитаты будет определять к какому роману она относится. Необходимо использовать модель "distilbert/distilroberta-base" из huggingface.
* Протестировать на valid: confusion_matrix, accuracy, f1. Сохранить эти метрики.

In [30]:
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


In [31]:
dataset = load_dataset("csv", data_files="train.csv")
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4512
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1128
    })
})

In [32]:
model_name = "distilbert/distilroberta-base"

In [33]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [34]:
tokenizer1 = AutoTokenizer.from_pretrained(model_name)

def tokenize(examples):
    return tokenizer1(examples["text"], padding="max_length", truncation=True)


In [35]:
dataset = dataset.map(tokenize, batched=True)
dataset

Map:   0%|          | 0/4512 [00:00<?, ? examples/s]

Map:   0%|          | 0/1128 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4512
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1128
    })
})

In [36]:
train_ds = dataset["train"]
valid_ds = dataset["test"]

In [ ]:
model1 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    cm = confusion_matrix(labels, predictions)
    return {"accuracy": acc, "f1": f1, "confusion_matrix": cm}

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    eval_steps=100,
    save_steps=100,
    logging_steps=100,
    push_to_hub=False
)

trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    compute_metrics=compute_metrics
)

trainer.train()


d:\Irisha\App\miniconda32\envs\dl_course\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,0.380200
200,0.250300
300,0.258900
400,0.232800
500,0.243400


TrainOutput(global_step=564, training_loss=0.2746572629779789, metrics={'train_runtime': 17686.4984, 'train_samples_per_second': 0.255, 'train_steps_per_second': 0.032, 'total_flos': 597692902735872.0, 'train_loss': 0.2746572629779789, 'epoch': 1.0})

In [ ]:
results = trainer.evaluate()

d:\Irisha\App\miniconda32\envs\dl_course\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:

print('Значения метрик:')
#print(results)
print(f"Accuracy =  {results['eval_accuracy']}, F1 =  {results['eval_f1']}")
print('Confusion_matrix:')
print(results['eval_confusion_matrix'])

Значения метрик:
Accuracy =  0.874113475177305, F1 =  0.8737487785846588
Confusion_matrix:
[[516  31]
 [111 470]]


2. **Претренировать модель с помощью unsupervised masked language modeling** на train-test.txt
* Воспользоваться вот этим туториалом https://huggingface.co/docs/transformers/main/tasks/masked_language_modeling
* Вывести метрики perplexity и loss до и после обучения

In [16]:
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
import math
import torch

In [3]:
with open('mlm.txt', 'r', encoding='utf-8') as file:
    lines = file.read()# file.readlines() #
print(lines[:100])

- Кровь! какую кровь? - встревожилась Пульхерия Александровна. - Под нижнюю подушку. - Благодарю-с..


In [4]:
def chanks (lines):
    lines = lines.split('. ')
    res = []
    for i in lines:
        k = i.split()
        res.extend([k[i:i + 6] for i in range(0, len(k), 6)])
    r = [' '.join(i) for i in res if len(i)>3]
    return r


In [5]:
dataset_mlm = Dataset.from_dict({"text": chanks (lines)})
dataset_mlm

Dataset({
    features: ['text'],
    num_rows: 63483
})

In [6]:
def tokenize(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

In [9]:
model_name = "distilbert/distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
dataset_mlm = dataset_mlm.map(tokenize, batched=True, remove_columns=["text"])
dataset_mlm

Map:   0%|          | 0/63483 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 63483
})

In [11]:
dataset_mlm = dataset_mlm.train_test_split(test_size=0.2, seed=42)
dataset_mlm

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 50786
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 12697
    })
})

In [12]:
model2 = AutoModelForMaskedLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
training_args_mlm = TrainingArguments(
    output_dir="mlm_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer_mlm = Trainer(
    model=model2,
    args=training_args_mlm,
    train_dataset=dataset_mlm["train"],
    eval_dataset=dataset_mlm["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/tmp/ipython-input-2318388377.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_mlm = Trainer(


In [18]:
eval_results = trainer_mlm.evaluate()
print('Значения до обучения:')
el = eval_results['eval_loss']
print(f"Perplexity: {math.exp(el):.2f}, Loss: {el:.4f}")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: osirvlad (osirvlad-hse-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Значения до обучения:
Perplexity: 4.23, Loss: 1.4422


In [19]:
trainer_mlm.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,1.005900,0.889371,0.001600


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,1.005900,0.889371,0.001600
2,0.853800,0.755954,0.001600
3,0.763500,0.669182,0.001600
4,0.705400,0.636488,0.001600
5,0.694400,0.611421,0.001600


TrainOutput(global_step=31745, training_loss=0.8475955594823311, metrics={'train_runtime': 7580.8102, 'train_samples_per_second': 33.496, 'train_steps_per_second': 4.188, 'total_flos': 5140682840741580.0, 'train_loss': 0.8475955594823311, 'epoch': 5.0})

In [25]:
print('Значения после обучения:')
eval_results = trainer_mlm.evaluate()
el = eval_results['eval_loss']
print(f"Perplexity: {math.exp(el):.2f}, Loss: {el:.4f}")

Значения после обучения:


Perplexity: 1.85, Loss: 0.6139


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
import os
directory = "/content/drive/My Drive/Irina/"

if not os.path.exists(directory):
    os.makedirs(directory)

In [48]:
torch.save(model2.state_dict(), directory + "model.pth")

In [49]:
model2.save_pretrained(directory + "model2")
trainer_mlm.save_model(directory + "trainer_mlm")

3. **Перетренировать классификатор из пункта 1**, но использовать претренированные веса из пункта 2
* сравнить метрики с пунктом 1

In [39]:
model3 = AutoModelForSequenceClassification.from_pretrained("trainer_mlm", num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at trainer_mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
from transformers import TrainingArguments, Trainer
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    cm = confusion_matrix(labels, predictions)
    return {"accuracy": acc, "f1": f1, "confusion_matrix": cm}

training_args3 = TrainingArguments(
    output_dir="results3",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_steps=100,
    save_steps=0,
    logging_steps=100,
    push_to_hub=False
)

trainer3 = Trainer(
    model=model3,
    args=training_args3,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    compute_metrics=compute_metrics
)

In [41]:
trainer3.train()

Step,Training Loss
100,0.409800
200,0.300100
300,0.277200
400,0.237900
500,0.231500
600,0.231700
700,0.227300
800,0.196700
900,0.228100
1000,0.188200


TrainOutput(global_step=1692, training_loss=0.22520758074226108, metrics={'train_runtime': 895.7795, 'train_samples_per_second': 15.111, 'train_steps_per_second': 1.889, 'total_flos': 1793078708207616.0, 'train_loss': 0.22520758074226108, 'epoch': 3.0})

In [50]:
results = trainer3.evaluate()
print('Значения метрик:')
print(f"Accuracy =  {results['eval_accuracy']}, F1 =  {results['eval_f1']}")
print('Confusion_matrix:')
print(results['eval_confusion_matrix'])

Trainer is attempting to log a value of "[[496  51]
 [ 51 530]]" of type <class 'numpy.ndarray'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Значения метрик:
Accuracy =  0.9095744680851063, F1 =  0.9095744680851063
Confusion_matrix:
[[496  51]
 [ 51 530]]


In [51]:
import pandas as pd

In [52]:
df = pd.read_csv("submission.csv")
text = df['text'].tolist()
text[:100]

['Он говорил ему, указывая на поля,',
 'Тревога беспредметная и бесцельная в настоящем,',
 '- Я о тебе, третьего дня',
 '- Что ж вы? - закричал',
 'Он с мучением задавал себе этот',
 '- Что ж шутить, - посмеиваясь',
 '- Не мучьте меня, говорите, говорите!',
 '- Ну, ну, голубчик, дядюшка, -',
 '3 го марта во всех комнатах',
 '1) различия в форме собственных имен:',
 'Князь Репнин назвал поручика Сухтелена.',
 '- Карагины, Жюли и Борис с',
 'Денисов ударил его по плечу, и',
 'Она всегда протягивала ему свою руку',
 '- Ну и довольно! - прервал',
 'Анатоль Курагин жил в Москве, потому',
 '"Если б я прежде видел ее',
 'Графиня Безухая, по справедливости, имела репутацию',
 'В остроге, в окружающей его среде,',
 '- Ну не выйду замуж, так',
 '- Когда это графиня приедет, измучила',
 '- Ну, в Америку собираться да',
 '- Я сам… с усам, -',
 '- Ноне их не разберешь, кто',
 '- начала было она петь... -',
 '- Она вымахалась, три угонки дала',
 '- Зарядили неаккуратно. Ничего! У вас',
 '- Да, был ш

In [54]:
inputs = tokenizer1(text, return_tensors="pt", padding=True, truncation=True)
inputs

{'input_ids': tensor([[    0, 25417, 17772,  ...,     1,     1,     1],
        [    0, 25417,  7258,  ...,     1,     1,     1],
        [    0,    12, 18697,  ...,     1,     1,     1],
        ...,
        [    0,    12, 18697,  ...,     1,     1,     1],
        [    0,    12, 18697,  ...,     1,     1,     1],
        [    0, 25417,  4333,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [57]:
inputs.to(device)

{'input_ids': tensor([[    0, 25417, 17772,  ...,     1,     1,     1],
        [    0, 25417,  7258,  ...,     1,     1,     1],
        [    0,    12, 18697,  ...,     1,     1,     1],
        ...,
        [    0,    12, 18697,  ...,     1,     1,     1],
        [    0,    12, 18697,  ...,     1,     1,     1],
        [    0, 25417,  4333,  ...,     1,     1,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [58]:
with torch.no_grad():
    logits = model3(**inputs).logits

In [59]:
len(text), logits

(1440,
 tensor([[-0.4345,  0.1506],
         [ 1.0411, -1.3237],
         [-4.6803,  4.5774],
         ...,
         [-4.7181,  4.5708],
         [-4.7072,  4.5784],
         [ 2.5747, -2.6114]], device='cuda:0'))

In [61]:
logits.shape

torch.Size([1440, 2])

In [62]:
predicted_class_id = logits.argmax(dim=1)
predicted_class_id

tensor([1, 0, 1,  ..., 1, 1, 0], device='cuda:0')

In [63]:
label = predicted_class_id.tolist()

In [64]:
data3 = {
    'text': text,
    'label': label
}
results_df = pd.DataFrame(data3)

results_df.head()

,text,label
0,"Он говорил ему, указывая на поля,",1
1,"Тревога беспредметная и бесцельная в настоящем,",0
2,"- Я о тебе, третьего дня",1
3,- Что ж вы? - закричал,1
4,Он с мучением задавал себе этот,1


In [65]:
results_df.to_csv(directory +"submission0.csv", index=False)